<a href="https://colab.research.google.com/github/shuklagauravjn/DMLAssignment1/blob/main/AssignmentDML1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
# Load CIFAR-10 dataset
(x_train, y_train), (x_val, y_val) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_val = x_train / 255.0, x_val / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_val = tf.keras.utils.to_categorical(y_val, 10)
# Load the ResNet50 model pre-trained on ImageNet, excluding the top fully connected layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of ResNet50 so they don't get retrained
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Checkpoint to save the best model
#checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
history = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint, early_stopping]
)
# Load the best saved model
model.load_weights('best_model.keras')

# Evaluate on the validation set
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
# Save the model
model.save('resnet50_cifar10_model.h5')

# Example of deploying with TensorFlow Serving or exporting the model
tf.saved_model.save(model, 'exported_model')
def predict_image(img, model):
    img_array = np.expand_dims(img, axis=0)
    predictions = model.predict(img_array)
    return predictions

# Example: Predicting on the first image in the validation set
predictions = predict_image(x_val[0], model)
print(f"Predicted class: {np.argmax(predictions)}")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 280s 347ms/step - accuracy: 0.2091 - loss: 2.2797 - val_accuracy: 0.3159 - val_loss: 1.8944
Epoch 2/50
 15/782 ━━━━━━━━━━━━━━━━━━━━ 4:44 371ms/step - accuracy: 0.3145 - loss: 1.9092